In [1]:
%matplotlib inline
import pandas as pd
import sklearn
from sklearn import *
from sklearn.preprocessing import LabelEncoder,LabelBinarizer,OneHotEncoder,Imputer,StandardScaler,MinMaxScaler
import numpy as np
import pickle
import os
from scipy import sparse
import scipy    
import scipy.sparse as sp
from IPython import display

#Not all data could be standardNormalized, Nonnegative MF could be not standarlized!!use min-max instead please

In [71]:
StandardScaler?

In [2]:
#read five tables
df_coupon_train=pd.read_csv('input/coupon_list_train.csv')
df_coupon_test=pd.read_csv('input/coupon_list_test.csv')
df_coupon_buy=pd.read_csv('input/coupon_detail_train.csv')
df_coupon_visit=pd.read_csv('input/coupon_visit_train.csv')
df_user=pd.read_csv('input/user_list.csv')
df_sub=pd.read_csv('input/sample_submission.csv')

In [15]:
df_sub.T

,0,1,2,3,4,5,6,7,8,9,...,22863,22864,22865,22866,22867,22868,22869,22870,22871,22872
USER_ID_hash,0000b53e182165208887ba65c079fc21,00035b86e6884589ec8d28fbf2fe7757,0005b1068d5f2b8f2a7c978fcfe1ca06,000cc06982785a19e2a2fdb40b1c9d59,0013518e41c416cd6a181d277dd8ca0b,001acdee812a18acfd7509172bed5700,001fd7876e3aa29393537c6baf308e43,002383753c1e5d6305c8aff6f89e26d6,0025cae7997d25ea5cf8851bb099c798,002822059a01d895fad84f2f2ff5c1f1,...,ffe65332382f1f264d84fa85a44f564c,ffe7362865b91c287affaf8912ef4750,ffecfec48a5778f3e470d8a4fb804723,ffed1129803b26815be16d5531a27250,fff193f946ae515d405376cdf7895f78,fff1a623187cefd7a594e338709b0f40,fff4a076cfda6ff9dbe85e1cb678791b,fff970d2014c3e10a77e38d540239017,fffafc024e264d5d539813444cf61199,ffff56dbf3c782c3532f88c6c79817ba
PURCHASED_COUPONS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#analysis for data inconsistency
item_num_total,user_num_total=len(df_coupon_train.COUPON_ID_hash),len(df_user.USER_ID_hash)#dont' user unique uid_rating, otherwise the index will exceed
item_num_visit=len(df_coupon_visit.VIEW_COUPON_ID_hash.unique())
user_num_visit=len(df_coupon_visit.USER_ID_hash.unique())
item_num_buy=len(df_coupon_buy.COUPON_ID_hash.unique())
user_num_buy=len(df_coupon_buy.USER_ID_hash.unique())
(item_num_total,user_num_total),(item_num_visit,user_num_visit),(item_num_buy,user_num_buy)

((19413, 22873), (32628, 22805), (19368, 22782))

#Genearate the leakage user rating from bestSubmission

In [3]:

set_coupon_train=set(df_coupon_train.COUPON_ID_hash.unique())
set_coupon_buy=set(df_coupon_buy.COUPON_ID_hash.unique())
set_coupon_visit=set(df_coupon_visit.VIEW_COUPON_ID_hash.unique())
set_coupon_test=set(df_coupon_test.COUPON_ID_hash.unique())
set_user_buy=set(df_coupon_buy.USER_ID_hash.unique())
set_user=set(df_user.USER_ID_hash.unique())
set_user_visit=set(df_coupon_visit.USER_ID_hash.unique())
print 'user intesect user_visit is %s' %len(set_user.intersection(set_user_visit))
print 'user intesect user_buy is %s' %len(set_user.intersection(set_user_buy))
print 'user num %s' %len(set_user)
print (set_user_visit.union(set_user_buy)).issubset(set_user)
print 'critical user num %s' %len(set_user.difference(set_user_visit.union(set_user_buy)))
set_user_leakage=set_user.difference(set_user_buy)#these users are not included in training
series_user_leakage=pd.Series(list(set_user_leakage))
df_others_submission=pd.read_csv('bestSubmission.csv')
df_leakage=df_others_submission.set_index('USER_ID_hash').loc[series_user_leakage].reset_index()

user intesect user_visit is 22805
user intesect user_buy is 22782
user num 22873
True
critical user num 40


In [5]:
#df_coupon_buy is contained in set_coupon_train
print set_coupon_train.issuperset(set_coupon_buy)
print set_coupon_train.issubset(set_coupon_visit)
print 'coupon_train and coupon_visit intersect by %s'%len(set_coupon_train.intersection(set_coupon_visit))
print 'coupon_train and coupon_buy intersect by %s' %len(set_coupon_train.intersection(set_coupon_buy))
print 'coupon_test and coupon_buy intersect by %s' %len(set_coupon_test.intersection(set_coupon_buy))
print 'coupon_test and coupon_visit intersect by %s'%len(set_coupon_test.intersection(set_coupon_visit))
print '#coupon_train and coupon_test intersect by %s'%len(set_coupon_train.intersection(set_coupon_test))
print '#coupon_visit not in coupon_train %s'%len(set_coupon_visit.difference(set_coupon_train))


True
False
coupon_train and coupon_visit intersect by 19412
coupon_train and coupon_buy intersect by 19368
coupon_test and coupon_buy intersect by 0
coupon_test and coupon_visit intersect by 39
#coupon_train and coupon_test intersect by 0
#coupon_visit not in coupon_train 13216


In [ ]:
#all_user_index=df_user.index # all the users now has a unique index
#part_coupon_index_train=df_coupon_train.index # all the coupons in training set now has a unique index
#uid_rating=df_user.set_index('USER_ID_hash').loc[df_rating['USER_ID_hash']]['user_global_index'].values
#qid_rating=df_coupon_train.set_index('COUPON_ID_hash').loc[df_rating['COUPON_ID_hash']]['coupon_global_index'].values

#Generate MAP from coupon_id to index

In [8]:
df_MAP_user=pd.DataFrame(df_coupon_buy.USER_ID_hash.unique(),columns=['USER_ID_hash'])
df_MAP_user['user_idx']=df_MAP_user.index
df_MAP_coupon=pd.DataFrame(df_coupon_buy.COUPON_ID_hash.unique(),columns=['COUPON_ID_hash'])
df_MAP_coupon['coupon_idx']=df_MAP_coupon.index

df_testsetMAP_Coupon=pd.DataFrame(df_coupon_test.COUPON_ID_hash)

#Generate Xu

In [9]:
df_rating=df_coupon_buy[[u'USER_ID_hash',u'COUPON_ID_hash','ITEM_COUNT']]
df_rating['ITEM_COUNT'].apply(lambda x : 1 if x >0 else x)

#WITH merge, one could generate repetitive index for coupon and user to have sparse matrix
df_rating=df_rating.merge(df_MAP_user,on='USER_ID_hash',how='left').merge(df_MAP_coupon,on='COUPON_ID_hash',how='left')

V=df_rating.ITEM_COUNT
qid_rating_repetitive=df_rating.coupon_idx
uid_rating_repetitive=df_rating.user_idx
Xu_train=sparse.coo_matrix((V,(qid_rating_repetitive,uid_rating_repetitive)),shape=(len(np.unique(qid_rating_repetitive)),len(np.unique(uid_rating_repetitive)))) # the user_id looks like (1,1,1,2,2,3,4,4,...)
#size of item-user should be (19413, 22873),but in df_coupon_buy, the size is only (19368, 22782)


In [27]:
#

In [10]:
script_path = os.path.abspath(os.path.dirname('__file__'))

import pandas as pd
import numpy as np
import os
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.feature_extraction import DictVectorizer as DV
from sklearn.preprocessing import StandardScaler

def extract_topn_item4user(df_itemUser_groupby_user, n=10, column="predict", merge_column="COUPON_ID_hash"):
    '''
    get top n row
    :param pandas.DataFrame df:
    :param int n:
    :param str column:
    :rtype: pandas.DataFrame
    '''
    return " ".join(df_itemUser_groupby_user.sort_index(by=column)[-n:][merge_column])

def binarizeFrame(df):
    dic=df.T.to_dict().values()
    vectorizer = DV( sparse = False )
    vec_x_cat_train = vectorizer.fit_transform( dic)
    return vec_x_cat_train

class Get_Match_Pref(BaseEstimator, TransformerMixin):
    '''
    get user pref is match coupon area
    '''

    def get_feature_names(self):

        return [self.__class__.__name__]

    def fit(self, date_frame, y=None):
        '''
        fit

        :param pandas.DataFrame: all data
        :rtype: Get_Price_Rate
        '''

        return self

    def transform(self, date_frame):
        '''
        transform

        :param pandas.DataFrame: all data
        :rtype: array
        '''
        res_sr = date_frame["PREF_NAME"] == date_frame["ken_name"]

        return res_sr.as_matrix()[None].T.astype(np.float)

class Get_user_feature(BaseEstimator, TransformerMixin):
    
    def get_feature_names(self):

        return [self.__class__.__name__]

    def fit(self, data_frame, y=None):
        '''
        fit
        :param pandas.DataFrame: all data
        :rtype: Get_Price_Rate
        '''
        return self
    
    def transform(self, df_coupon_train):
        '''
        transform
        :param pandas.DataFrame: all data
        :rtype: array
        '''
        le=LabelEncoder()
        df_coupon_train['SEX_ID_Num']=le.fit_transform(df_coupon_train['SEX_ID'])
        return df_coupon_train[['SEX_ID_Num', 'AGE']].as_matrix().astype(np.float)


          
class Get_coupon_feature(BaseEstimator, TransformerMixin):
    def encode(self,v):
        u = v.unique()
        lm = len(u) + 1
        d = {np.nan:lm, "nan": lm}  
        for ii, iu in enumerate(u):
            d[iu] = ii
        return v.apply(lambda x: d.get(x, lm) ).values

    def binarizeCategorical(self,df_coupon_cate):
        oh=OneHotEncoder()
        res_final = None
        for i, col in enumerate(df_coupon_cate.columns):
            #print col
            res = self.encode(df_coupon_cate[col])
            #print res.shape, np.unique(res), 
            #print len(df_coupon_train_cat[col].unique())
            res = oh.fit_transform(res.reshape(len(res), 1))
            res_final = res if res_final is None else sp.hstack((res_final, res))
            #print res_final.shape
        return res_final.todense()
    
    
    def get_feature_names(self):

        return [self.__class__.__name__]

    def fit(self, df_coupon_train_cat, y=None):
        '''
        fit
        :param pandas.DataFrame: all data
        :rtype: Get_Price_Rate
        '''
        #df_coupon_train_cat.apply(lambda x: len(x.unique()))
        return self
    
    def transform(self, df_coupon):
        '''
        transform
        :param pandas.DataFrame: coupon property pandas data frame
        :rtype: array
        '''
        #'COUPON_ID_hash',
        df_coupon_nume=df_coupon[['PRICE_RATE', 'CATALOG_PRICE','DISCOUNT_PRICE','VALIDPERIOD','DISPPERIOD',\
                                              'USABLE_DATE_MON', 'USABLE_DATE_TUE',\
               'USABLE_DATE_WED', 'USABLE_DATE_THU', 'USABLE_DATE_FRI',\
               'USABLE_DATE_SAT', 'USABLE_DATE_SUN', 'USABLE_DATE_HOLIDAY','USABLE_DATE_BEFORE_HOLIDAY']]
        im=Imputer()
        mm=MinMaxScaler()
        ss=StandardScaler(with_mean=False)
        df_coupon_nume_im=pd.DataFrame(im.fit_transform(df_coupon_nume),columns=df_coupon_nume.columns,index=df_coupon_nume.index)
        df_coupon_nume_norm=pd.DataFrame(ss.fit_transform(df_coupon_nume_im),columns=df_coupon_nume.columns,index=df_coupon_nume.index)
        df_coupon_nume_norm=pd.DataFrame(mm.fit_transform(df_coupon_nume_norm),columns=df_coupon_nume.columns,index=df_coupon_nume.index)
        ############################################
        df_coupon_cate=df_coupon[['CAPSULE_TEXT', 'GENRE_NAME', 'large_area_name', 'ken_name','small_area_name']]
        #sparse_m=binarizeCategorical(df_coupon_train_cat)
        #sp.hstack(sparse_m,df_coupon_train_num.values)
        df_coupon_binary=pd.DataFrame(self.binarizeCategorical(df_coupon_cate))  
        df_coupon_combine=df_coupon_nume_norm.join(df_coupon_binary)
        #df_coupon_combine.set_index('COUPON_ID_hash',inplace=True)
        #return df_coupon_combine.as_matrix().astype(np.float)
        return df_coupon_combine


#generate Xs, and persistentate the Xs,Xu

In [11]:
# make sure that the transformation is done to the whole 
fu_obj = FeatureUnion(transformer_list=[('item_feature', Get_coupon_feature())])
train_part_coupon=df_coupon_train.set_index('COUPON_ID_hash').loc[df_MAP_coupon.COUPON_ID_hash].reset_index()

Xs=fu_obj.fit_transform(pd.concat([train_part_coupon,df_coupon_test],axis=0)) # df_coupon_train is already the natural global index for items
Xs_train=Xs[0:train_part_coupon.shape[0],:]
Xs_test=Xs[train_part_coupon.shape[0]:,:]
print Xs_train.shape,Xu_train.shape,Xs_test.shape
with open('Xstr_Xutr_Xste.pickle', 'wb') as f:
        pickle.dump((Xs_train,Xu_train,Xs_test),f)

(19368, 163) (19368, 22782) (310, 163)


In [ ]:
'''
#transform the hash code to natural index in df_coupon_train
df_coupon_visit.drop(labels=[u'I_DATE',u'PAGE_SERIAL',u'REFERRER_hash',u'SESSION_ID_hash',u'PURCHASEID_hash'],axis=1,inplace=True)
#g_visit=df_coupon_visit.groupby('VIEW_COUPON_ID_hash')
#g_visit.count()
g_visit2=df_coupon_visit.groupby(['VIEW_COUPON_ID_hash','USER_ID_hash'])
#df_rating=pd.DataFrame(g_visit2.apply(lambda x: np.sum(x['PURCHASE_FLG'])))
df_rating=df_coupon_visit[[u'USER_ID_hash',u'VIEW_COUPON_ID_hash','PURCHASE_FLG']] #duplicate coupon??????
df_user['user_global_index']=df_user.index # all the users now has a unique index
df_coupon_train['coupon_global_index']=df_coupon_train.index # all the coupons in training set now has a unique index
#df_CouponProperty=pd.concat([df_coupon_train,df_coupon_test])
# build up user global index and coupon global index for row and column index for sparse matrix
user_id=df_user.set_index('USER_ID_hash').loc[df_rating['USER_ID_hash']]['user_global_index'].values
coupon_id=df_coupon_train.set_index('COUPON_ID_hash').loc[df_rating['VIEW_COUPON_ID_hash']]['coupon_global_index'].values
num_item,num_user=len(df_coupon_train.COUPON_ID_hash.unique()),len(df_user.USER_ID_hash.unique())
V=df_rating.PURCHASE_FLG
Xu=sparse.coo_matrix((V,(coupon_id,user_id)),shape=(num_item,num_user)) # the user_id looks like (1,1,1,2,2,3,4,4,...)
#df_coupon_visit2=df_coupon_visit.groupby(['USER_ID_hash','VIEW_COUPON_ID_hash'])['PURCHASE_FLG'].agg({'PURCHASE_FLG_sum':'sum'})
#df_rating=pd.DataFrame(g_visit2.agg(lambda x: np.sum(x['PURCHASE_FLG'])))
'''

# transform the user-{NEW_item} rate matrix into submission form

In [19]:
df_rt=pd.read_csv('user_item_rating.csv',header=None)
df_rt=df_rt.set_index(df_MAP_user.USER_ID_hash)
df_rt.columns=df_testsetMAP_Coupon.COUPON_ID_hash

In [30]:
#df_sub.set_index(userindex,inplace=True)
def top_series(s, num):
    tmp = pd.Series(s.order(ascending=False)[:num].index.tolist(), index = range(num))
    return tmp
dfdot_genre_top = df_rt.T.apply(lambda x: top_series(x, 10))

In [31]:
df_final=dfdot_genre_top.T.apply((lambda x: " ".join(x)), axis=1).to_frame()

In [34]:
df_final=df_final.reset_index()

df_final.columns=['USER_ID_hash','PURCHASED_COUPONS']

In [40]:
pd.concat([df_final,df_leakage],axis=0).to_csv('sub2.csv',index=False)

# Generate user-coupon feature for FM/rf, but have memory problem, Xtrain is too big

In [7]:
#generate feature for factorization machine user_[0,0,0,1,.....]_movie[1,0,0,0,0.....]_othermovies_[0.333,0.333,0.333,....][timeStamp]last_bought[0,1]-[target]

#Generate training set
df_coupon_transaction=df_coupon_visit# never modify the original one
join_df = pd.merge(df_coupon_transaction, df_coupon_train,left_on="VIEW_COUPON_ID_hash", right_on="COUPON_ID_hash")
join_df = pd.merge(join_df, df_user,left_on="USER_ID_hash", right_on="USER_ID_hash")
feature_list = [('item_feature', Get_coupon_feature()),('user_item',Get_Match_Pref()),('getUserFeature',Get_user_feature())]
fu_obj = FeatureUnion(transformer_list=feature_list)
X_train = fu_obj.fit_transform(join_df)
y_train = join_df["PURCHASE_FLG"]
assert X_train.shape[0] == y_train.size




In [8]:
#Generate test set
#generate the Cartesian Product
df_coupon_test["cross"] = 1 # all test items are new, ie, cold-start items
df_user["cross"] = 1 # all users are old
test_df_item_user = pd.merge(df_coupon_test,df_user, on="cross")# Cartesian product
# create test Feature
X_test = fu_obj.transform(test_df_item_user)
# predict test data

In [9]:
# fit model
from sklearn import ensemble
original_params = {'n_estimators': 1000,'n_jobs':-1}
params = dict(original_params)
clf=ensemble.RandomForestClassifier(**params)
#clf = ensemble.GradientBoostingClassifier(**params)
clf.fit(X_train, y_train)
# create test_df

#prediction


predict_proba = clf.predict_proba(X_test)
pos_idx = np.where(clf.classes_ == True)[0][0] # find out which column is positive
test_df_item_user["predict"] = predict_proba[:, pos_idx] 

top10_coupon=test_df_item_user.groupby("USER_ID_hash").apply(extract_topn_item4user) #for each user
top10_coupon.name = "PURCHASED_COUPONS"
top10_coupon.to_csv("submission_rf.csv", header=True)

KeyError: '<ipython-input-1-3cdaee5bf3a1>'

# playground for sparse

In [ ]:
from scipy import sparse
from scipy.sparse import spdiags
data = np.array([[1, 2, 3, 4], [1, 2, 3, 4], [1, 2, 3, 4]])
diags = np.array([0, -1, 2])
data*data
np.multiply(data,data)
data,sp*data
1. / (np.sqrt(np.sum(np.multiply(data,data), axis=1)))
sp=sparse.spdiags(1./np.sqrt(np.sum(data * data, axis=1)),0,data.shape[0],data.shape[0])
sp.todense()*data
np.sum(data,axis=1)
####################################
sparse.coo_matrix.multiply(Xu,Xu.toarray())

Xu.sum(1).shape
(1. / (np.sqrt(Xu.multiply(Xu).sum(1)) )).shape
1. / np.sqrt((Xu.multiply(Xu).sum(1)) ).T
sparse.spdiags((1. / np.sqrt((Xu.multiply(Xu).sum(1)) )).T, 0, Xu.shape[0], Xu.shape[0]).todense()
sparse.spdiags?

row  = np.array([0, 3, 1, 0])
col  = np.array([0, 3, 1, 2])
data = np.array([4, 5, 7, 9])
A=sparse.coo_matrix((data, (row, col)), shape=(4, 4))


row = np.array([0, 2, 2, 0 ])
col = np.array([0, 0, 1, 2])
data = np.array([1.0, 2.0, 3.0, 4.0])
B=sparse.csc_matrix((data, (row, col)), shape=(4, 4))

AA=sparse.csc_matrix(A)

BB=sparse.csc_matrix(B)

sparse.csc_matrix.multiply(AA,BB.T).todense()

BB.todense(),AA.todense()

In [ ]:
BB.shape

In [ ]:
(BB+BB).todense(),BB.toarray()

In [ ]:
scipy.__version__

In [ ]:
reload(scipy)

In [ ]:
np.maximum(BB.toarray(),10)

In [ ]:
sparse.csc_matrix.maximum(BB,1).toarray()

In [ ]:
 print("Hu is %s" %BB)

In [ ]:
BB.diagonal()

In [ ]:
np.sum(BB.diagonal())

In [ ]:
df_coupon_test

In [ ]:
Xs_test.shape

In [ ]:
import lce

In [ ]:
import pickle
with open('Xs_test.pickle', 'wb') as f:
    pickle.dump(Xs_test, f)

In [17]:
df_rating.cumsum?